## Experiments with GPT-Neo

In [1]:
!pip install transformers

In [2]:
#
# NOTE
# Comment out the following code if you are not running this notebook on NLULab (Azure).
#

#import os

#cache_dir = '/home/jovyan/nlu2/.cache/transformers'
#os.environ['TRANSFORMERS_CACHE'] = cache_dir

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    gpu_stat = torch.cuda.device_count(), device
else:
    device = -1
    gpu_stat = 'no-gpu'

gpu_stat

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1614378098133/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


'no-gpu'

### Model Size Chart

|Size|Parameters|
|--- | ---|
|S|125M params|
|M|1.3B params|
|L|2.7B params|

In [4]:
model_size = "L"  # Can be S, M, L

In [5]:
from transformers import pipeline

In [6]:
%%time


def load_model(msize):
    model_name = 'EleutherAI/gpt-neo-125M'
    if msize == "M":
        model_name = 'EleutherAI/gpt-neo-1.3B'
    elif msize == "L":
        model_name = 'EleutherAI/gpt-neo-2.7B'
    return pipeline('text-generation', model=model_name, device=device)

print("Loading model (this might take a while)...")
gpt_neo = load_model(model_size)

Loading model (this might take a while)...
CPU times: user 45.9 s, sys: 8.23 s, total: 54.2 s
Wall time: 1min 57s


In [7]:
def complete(prompt, temperature=0.9, do_sample=True, min_length=1, max_length=160, stop=["\n"], top_k=1):
    prompt = prompt.rstrip()
    _max_length = int(len(prompt.split())*1.5 + max_length)
    choices = gpt_neo(
        prompt, 
        temperature=temperature,
        do_sample=do_sample,
        min_length=min_length,
        max_length=_max_length,
        top_k=top_k
    )
    result = list()
    for i in range(min(top_k, len(choices))):
        output = choices[i].get('generated_text', '')
        if output.startswith(prompt):
            output = output[len(prompt):]
        for s in stop:
            _idx = output.find(s)
            if _idx > -1:
                output = output[:_idx]
        result.append(output.strip())
    if len(result) == 1:
        result = result[0]
    return result

#### Text Completion

In [8]:
%%time

prompt = "As a company Ericsson is the"
complete(prompt, max_length=60, top_k=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 1min 27s, sys: 161 ms, total: 1min 27s
Wall time: 21.9 s


'world’s leading mobile telecommunications equipment and service provider. Ericsson is a global company with headquarters in Stockholm, Sweden, and operates in more than 200 countries and jurisdictions worldwide. With more than 5,600 employees, Ericsson is a leading global technology company. The Ericsson Group has approximately 30,000'

#### Question Answering (no context)

In [9]:
%%time

prompt = """
Q: What is upload data-rate in LTE?
A: 75 Mbps

Q: What is EPS?
A: Evolved Packet System

Q: What is peak download rate for 4G?
A:
"""

complete(prompt, stop=["\n", "Q:"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 4min 50s, sys: 978 ms, total: 4min 51s
Wall time: 1min 12s


'Up to 150 Mbps'

#### SQL Generation

In [10]:
%%time

prompt = """
Q: List the names of employees at Ericsson?
SQL: SELECT name FROM employees WHERE company = 'Ericsson';

Q: Show the highest paid player at the NBA?
SQL: SELECT * FROM players WHERE league = 'NBA' ORDER BY salary DESC LIMIT 1;

Q: How many employees are there at GAIA?
SQL: 
"""

complete(prompt, stop=["\n", "SQL:"], temperature=0.6)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 5min 33s, sys: 808 ms, total: 5min 34s
Wall time: 1min 23s


'SELECT COUNT(*) FROM employees;'

#### code generation 1

In [11]:
prompt = """
Generate python code for image classification

import tensorflow as tf
"""

#complete(prompt, stop=[])
code = complete(prompt, stop=[])
print(code)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


import numpy as np

# Load the image
img = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])

# Convert to numpy array
img = tf.image.decode_jpeg(img, channels=3)

# Convert to numpy array
img = tf.image.convert_image_dtype(img, tf.float32)

# Convert to numpy array
img = tf.image.convert_image_dtype(img, tf.uint8)

# Convert to numpy array
img = tf.image.convert_image_dtype(img, tf.uint8)

# Convert to numpy


#### pandas code generation

In [12]:
import pandas as pd
import numpy as np

In [13]:
df = pd.DataFrame( {"Gender": ["boy","boy","boy","boy","boy","girl","girl","girl","girl"],
                   "Division": ["one","one","one","two","two","one","one","two","two"], 
                   "Marks": [50,55,67,85,44,84,65,56,87]} )

In [14]:
df

,Gender,Division,Marks
0,boy,one,50
1,boy,one,55
2,boy,one,67
3,boy,two,85
4,boy,two,44
5,girl,one,84
6,girl,one,65
7,girl,two,56
8,girl,two,87


In [15]:
prompt = """
we have dataframe with fields "Gender", "Division" and "Marks".

Q: 'How many unique values in Division Column?'
A: 'df["Division".nunique()]';

Q: 'Find the Division of boy who scored 55 marks.'
A: 'df.loc[(df.loc[:, "Gender"] == "boy") & (df.loc[:, "Marks"] == 55), "Division"]';

Q: 'Find the average Marks scored by Girls.'
A: 'np.mean(df.loc[(df.loc[:, "Gender"] == "girl"), "Marks"])'

Q: 'Display Division of girl who scored maximum marks.'
A: 
"""

In [16]:
complete(prompt, stop=["\n", "A:"], temperature=0.6)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'\'df.loc[(df.loc[:, "Gender"] == "girl") & (df.loc[:, "Marks"] == max(df.loc[:, "Marks"])), "Division"]\''

In [17]:
df.loc[(df.loc[:, "Gender"] == "girl") & (df.loc[:, "Marks"] == max(df.loc[:, "Marks"])), "Division"]

8    two
Name: Division, dtype: object

In [18]:
prompt = """
we have dataframe with fields "Gender", "Division" and "Marks".

Q: 'How many unique values in Division Column?'
A: 'df["Division".nunique()]';

Q: 'Find the Division of boy who scored 55 marks.'
A: 'df.loc[(df.loc[:, "Gender"] == "boy") & (df.loc[:, "Marks"] == 55), "Division"]';

Q: 'Find the average Marks scored by Girls.'
A: 'np.mean(df.loc[(df.loc[:, "Gender"] == "girl"), "Marks"])'

Q: 'Display Division of girl who scored maximum marks.'
A: 'df.loc[(df.loc[:, "Gender"] == "girl") & (df.loc[:, "Marks"] == max(df.loc[:,"Marks"])), "Division"]'

Q: 'Find the median Marks scored by Boys.'
A: 
"""

In [19]:
complete(prompt, stop=["\n", "A:"], temperature=0.6)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'\'np.median(df.loc[(df.loc[:, "Gender"] == "boy"), "Marks"])\''

In [20]:
np.median(df.loc[(df.loc[:, "Gender"] == "boy"), "Marks"])

55.0

#### code generation 2

In [21]:
prompt = """
def is_palindrome(s):
    '''Check whether a string is a palindrome'''
    return 
"""

In [22]:
complete(prompt, stop=["\n"], temperature=0.6)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'all(s == s[::-1] for s in s)'

#### summary

In [23]:
prompt = """
These latest enhancements to Ericsson’s portfolio were developed in
close collaboration with customers such as Verizon. They will enable
service providers to seamlessly evolve towards cloud-native technologies
and open network architectures and meet the demand for more deployment
flexibility. Service providers will also be able to extend Ericsson
Cloud RAN reach with seven million 5G-ready Ericsson Radio System radios
already in the field globally.
\"\"\"
Summarize this paragraph:
\"\"\"
"""

In [24]:
temp = complete(prompt, stop=["\"\"\"", "."], max_length= 60, temperature=0.9, top_k = 3)
print(temp)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The new Ericsson 5G-ready radio system is the world’s most
advanced 5G-ready radio system


#### Java script

In [34]:
prompt = "Use list comprehension to convert this into one line of JavaScript:\n\ndogs.forEach((dog) => {\n    car.push(dog);\n});\n\nJavaScript one line version:"

In [42]:
temp = complete(prompt, stop=[";"], max_length= 100, temperature=0.01, top_k = 4)
print(temp)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 258, but ``max_length`` is set to 202.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


In [27]:
prompt = "How to define a name function in JavaScript?JavaScript example:"

In [28]:
temp = complete(prompt, stop=["}"], max_length= 100, temperature=0.5, top_k = 1)
print(temp)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


function name(name) {

return name;


In [29]:
prompt = "How can you create an Array in JavaScript?JavaScript example:"

In [30]:
temp = complete(prompt, stop=[";"], max_length= 100, temperature=0.5, top_k = 1)
print(temp)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


var myArray = []


#### k8s

In [31]:
prompt = """
Generate Kubernetes Manifests from requirements.

Q: 'Deploy the wordpress project with 2 replicas.'
A: 'apiVersion: apps/v1\n\tkind: Deployment metadata: name: wordpress-deployment spec: replicas: 2'      

Q: 'Deploy nginx with 3 replicas.'
A: 'apiVersion: apps/v1  kind: Deployment metadata: name: nginx-deployment labels: app: nginx spec: replicas: 3'

Q: 'Deploy mongodb with two replicas.'
A: 
"""

In [32]:
complete(prompt,max_length=100, temperature=0.6)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"'apiVersion: apps/v1  kind: Deployment metadata: name: mongodb-deployment spec: replicas: 2'"

In [36]:
prompt = """
Input: expose wordpress on port 80
Output:
apiVersion: v1
kind: Service
metadata:
  name: wordpress-service
spec:
  selector:
    app: wordpress
  ports:
    - protocol: TCP
      port: 80

Input: deploy wordpress with 2 replicas
Output:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: wordpress-deployment
  labels:
    app: wordpress
spec:
  replicas: 3
  selector:
    matchLabels:
      app: wordpress
  template:
    metadata:
      labels:
        app: wordpress
    spec:
      containers:
      - name: wordpress
        image: wordpress:3.5.0
        ports:
        - containerPort: 80

Input: deploy Elasticsearch
Output:
"""

In [41]:
temp = complete(prompt,stop = ["\n\n"], max_length=300, temperature=0.1)
print(temp)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


apiVersion: apps/v1
kind: Deployment
metadata:
  name: elasticsearch-deployment
  labels:
    app: elasticsearch
spec:
  replicas: 3
  selector:
    matchLabels:
      app: elasticsearch
  template:
    metadata:
      labels:
        app: elasticsearch
    spec:
      containers:
      - name: elasticsearch
        image: elasticsearch:2.4.5
